## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = '/content/drive/MyDrive/Colab Notebooks/데이콘_lg/'

# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [ ]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [ ]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [ ]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [ ]:
# errtype_pick = [16, 26, 31, 40, 15, 5, 23, 14, 22, 34, 4, 12, 17, 42, 11, 33]

In [ ]:
# train_err['errtype_pick'] = train_err['errtype'].isin(errtype_pick).astype('int64')

In [ ]:
# train_err = train_err[train_err.errtype_pick == 1]
# train_err

In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [00:48<00:00, 338257.43it/s]


(15000, 42)

In [ ]:
train_err.dropna(axis = 0, inplace=True)
train_err.shape

(16554662, 6)

In [ ]:
items = train_err.errcode.unique().astype(str)
items

array(['1', '0', '2', ..., '4526', '3965', '25999'], dtype='<U39')

In [ ]:
len(train_err.errcode.unique())

2805

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(train_err.errcode)
labels.shape

(16554662,)

In [ ]:
train_err['errcode_label'] = labels

In [ ]:
errcode_pick = [2, 4, 279, 2728, 196, 2796, 2755, 
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800,2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153]

In [ ]:
train_err['errcode_pick'] = train_err['errcode_label'].isin(errcode_pick).astype('int64')

In [ ]:
train_err.errcode_pick.value_counts()

1    16194360
0      360302
Name: errcode_pick, dtype: int64

In [ ]:
train_err = train_err[train_err.errcode_pick == 1]
train_err

user_id            time model_nm  ... errcode  errcode_label errcode_pick
0           10000  20201101025616  model_3  ...       1              4            1
1           10000  20201101030309  model_3  ...       1              4            1
2           10000  20201101030309  model_3  ...       1              4            1
3           10000  20201101050514  model_3  ...       1              4            1
4           10000  20201101050515  model_3  ...       0              2            1
...           ...             ...      ...  ...     ...            ...          ...
16554658    24999  20201130163051  model_3  ...       1              4            1
16554659    24999  20201130172625  model_3  ...       1              4            1
16554660    24999  20201130172625  model_3  ...       0              2            1
16554661    24999  20201130172631  model_3  ...       0              2            1
16554662    24999  20201130210625  model_3  ...       1              4            1

[16194360 rows x 8 columns]

In [ ]:
train_err.errcode_label.value_counts().sort_index()

2       2599123
4       8906967
176       12040
196      263577
279      166382
297         982
1209      20249
1537       7335
2153       7392
2492       5028
2500     332422
2517       9317
2518     334018
2579     129876
2598       4023
2639      65992
2728     897863
2729        172
2744        163
2755     104138
2758        105
2763        157
2768      11959
2771      34631
2774        138
2776       1583
2785       3914
2791       1740
2796     219238
2798     100940
2799    1835600
2800       4011
2802        392
2803     110370
2804       2523
Name: errcode_label, dtype: int64

In [ ]:
id_errcode = train_err[['user_id','errcode_label']].values
errcode = np.zeros((train_user_number,2805))

for person_idx, code in tqdm(id_errcode):
    # person_idx - train_user_id_min 위치에 person_idx, fwver에 해당하는 error값을 +1
    errcode[person_idx - train_user_id_min,code] += 1
errcode.shape

100%|██████████| 16194360/16194360 [00:42<00:00, 380908.33it/s]


(15000, 2805)

In [ ]:
type_code = np.append(error, errcode, axis=1)
type_code.shape

(15000, 2847)

In [ ]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.2 problem

In [ ]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련

In [ ]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = type_code
train_y = problem
del type_code, problem
print(train_x.shape)
print(train_y.shape)

(15000, 2847)
(15000,)


In [ ]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.806029	valid_0's pr_auc: 0.78991
[40]	valid_0's auc: 0.809146	valid_0's pr_auc: 0.803905
[60]	valid_0's auc: 0.81079	valid_0's pr_auc: 0.80611
[80]	valid_0's auc: 0.809516	valid_0's pr_auc: 0.805302
[100]	valid_0's auc: 0.806315	valid_0's pr_auc: 0.803632
[120]	valid_0's auc: 0.804536	valid_0's pr_auc: 0.801559
[140]	valid_0's auc: 0.803132	valid_0's pr_auc: 0.800633
Early stopping, best iteration is:
[54]	valid_0's auc: 0.811496	valid_0's pr_auc: 0.806445
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.811565	valid_0's pr_auc: 0.260404
[40]	valid_0's auc: 0.820586	valid_0's pr_auc: 0.369059
[60]	valid_0's auc: 0.818794	valid_0's pr_auc: 0.387802
[80]	valid_0's auc: 0.81655	valid_0's pr_auc: 0.378266
[100]	valid_0's auc: 0.813929	valid_0's pr_auc: 0.383697
[120]	valid_0's auc: 0.81138	valid_0's pr_auc: 0.382572
[140]	valid_0's auc: 0.808841	valid_0's pr_auc: 0.38168

# 4. 교차검증 점수 확인

In [ ]:
print(np.mean(auc_scores))

0.8136055053086977


# 5. 제출 파일 생성

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

user_id            time model_nm       fwver  errtype errcode
0    30000  20201101030227  model_1  04.16.3553       31       1
1    30000  20201101030227  model_1  04.16.3553       33       2
2    30000  20201101030228  model_1  04.16.3553       15       1
3    30000  20201101030256  model_1  04.16.3553       22       1
4    30000  20201101030300  model_1  04.16.3553       11       1

In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
test_err.dropna(axis = 0, inplace=True)
test_err.shape

(16532644, 6)

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532644/16532644 [00:50<00:00, 330590.68it/s]

(14999, 42)


In [ ]:
test_err

user_id            time model_nm       fwver  errtype errcode
0           30000  20201101030227  model_1  04.16.3553       31       1
1           30000  20201101030227  model_1  04.16.3553       33       2
2           30000  20201101030228  model_1  04.16.3553       15       1
3           30000  20201101030256  model_1  04.16.3553       22       1
4           30000  20201101030300  model_1  04.16.3553       11       1
...           ...             ...      ...         ...      ...     ...
16532643    44998  20201130210050  model_1  04.16.3553       40       0
16532644    44998  20201130211831  model_1  04.16.3553       31       1
16532645    44998  20201130211832  model_1  04.16.3553       15       1
16532646    44998  20201130212259  model_1  04.16.3553       16       1
16532647    44998  20201130212316  model_1  04.16.3553       31       0

[16532644 rows x 6 columns]

In [ ]:
a = set(test_err.errcode).intersection(train_err.errcode)
print(a)

{'terminate by peer user', 'S-64002', 'scanning timeout', 'Q-64002', '13', 'S-61001', '0', '4', 'H-51046', 'B-A8002', 'P-41007 ', 'standby', 'connection timeout', 'C-11017', 'connectionterminated by local host', 'NFANDROID2', 'connection fail to establish', '2', '6', '84', '21', '80', 'V-21008', 'PHONE_ERR', '79', '86', 'UNKNOWN', '1', '90', '5', 'S-65002', '78', '8.0', 'active', '14'}


In [ ]:
a = list(a)
print(a)

['terminate by peer user', 'S-64002', 'scanning timeout', 'Q-64002', '13', 'S-61001', '0', '4', 'H-51046', 'B-A8002', 'P-41007 ', 'standby', 'connection timeout', 'C-11017', 'connectionterminated by local host', 'NFANDROID2', 'connection fail to establish', '2', '6', '84', '21', '80', 'V-21008', 'PHONE_ERR', '79', '86', 'UNKNOWN', '1', '90', '5', 'S-65002', '78', '8.0', 'active', '14']


In [ ]:
test_err.errcode.unique()

array(['1', '2', 'connection timeout', ..., '5803', '5444', '5691'],
      dtype=object)

In [ ]:
test_err['errcode_pick'] = test_err['errcode'].isin(a).astype('int64')

In [ ]:
test_err.errcode_pick.value_counts()

1    16186888
0      345756
Name: errcode_pick, dtype: int64

In [ ]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errtype  errcode errcode_pick
0           30000  20201101030227  model_1  ...      31        1            1
1           30000  20201101030227  model_1  ...      33        2            1
2           30000  20201101030228  model_1  ...      15        1            1
3           30000  20201101030256  model_1  ...      22        1            1
4           30000  20201101030300  model_1  ...      11        1            1
...           ...             ...      ...  ...     ...      ...          ...
16532643    44998  20201130210050  model_1  ...      40        0            1
16532644    44998  20201130211831  model_1  ...      31        1            1
16532645    44998  20201130211832  model_1  ...      15        1            1
16532646    44998  20201130212259  model_1  ...      16        1            1
16532647    44998  20201130212316  model_1  ...      31        0            1

[16186888 rows x 7 columns]

In [ ]:
labels = encoder.transform(test_err.errcode)
labels.shape

(16186888,)

In [ ]:
test_err['errcode_label'] = labels

In [ ]:
errcode_pick = [2, 4, 279, 2728, 196, 2796, 2755, 
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800,2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153]

In [ ]:
train_err['errcode_pick'] = train_err['errcode_label'].isin(errcode_pick).astype('int64')

In [ ]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errcode  errcode_pick errcode_label
0           30000  20201101030227  model_1  ...       1             1             4
1           30000  20201101030227  model_1  ...       2             1           279
2           30000  20201101030228  model_1  ...       1             1             4
3           30000  20201101030256  model_1  ...       1             1             4
4           30000  20201101030300  model_1  ...       1             1             4
...           ...             ...      ...  ...     ...           ...           ...
16532643    44998  20201130210050  model_1  ...       0             1             2
16532644    44998  20201130211831  model_1  ...       1             1             4
16532645    44998  20201130211832  model_1  ...       1             1             4
16532646    44998  20201130212259  model_1  ...       1             1             4
16532647    44998  20201130212316  model_1  ...       0             1             2

[16186888 rows x 8 columns]

In [ ]:
test_err.errcode_label.value_counts().sort_index()

2       2565532
4       8750154
176       11740
196      316110
279      155401
297         997
1209      19084
1537      12991
2153      13030
2492       5059
2500     324186
2517       9427
2518     326179
2579     140642
2598       3164
2639      58820
2728     855747
2729        211
2744        113
2755     108244
2758        142
2763        181
2768      11808
2771      35026
2774        136
2776       1636
2785       3571
2791       1064
2796     225519
2798     109690
2799    1890632
2800      84809
2802        347
2803     139947
2804       5549
Name: errcode_label, dtype: int64

In [ ]:
id_code = test_err[['user_id','errcode_label']].values
test_code = np.zeros((test_user_number,2805))

for person_idx, code in tqdm(id_code):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_code[person_idx - test_user_id_min, code] += 1
test_code = test_code.reshape(test_code.shape[0],-1)
print(test_code.shape)

100%|██████████| 16186888/16186888 [00:42<00:00, 381962.83it/s]

(14999, 2805)


In [ ]:
test_type_code = np.append(test_x, test_code, axis=1)
test_type_code.shape

(14999, 2847)

In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_type_code)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

array([[0.86967588],
       [0.21384194],
       [0.57708317],
       ...,
       [0.73280674],
       [0.86947281],
       [0.40035283]])

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
sample_submssion.to_csv("dacon_baseline_errcode_type_code_pick.csv", index = False)
sample_submssion

user_id   problem
0        30000  0.869676
1        30001  0.213842
2        30002  0.577083
3        30003  0.770127
4        30004  0.650945
...        ...       ...
14994    44994  0.171122
14995    44995  0.337495
14996    44996  0.732807
14997    44997  0.869473
14998    44998  0.400353

[14999 rows x 2 columns]